# 1. Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
import glob
import calendar
import geopandas as gpd
from shapely.geometry import Point
import json
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column
%matplotlib inline

# 2. Consolidation of Data

In [ ]:
df1= pd.DataFrame()
for i in glob.glob('Data/Crime_Statistics/*.csv'):
    df=pd.read_csv(i)
    df1=df1.append(df,ignore_index=True)
df1.to_csv('Data/Crime_data.csv')    

# 3. Initial Study of Dataset

In [ ]:
df_lib=pd.read_csv('Data/Crime_data.csv', index_col = 'Reported Date', parse_dates=True)
df_lib.drop(columns=['Unnamed: 0','Offence Level 1 Description','Offence Level 3 Description'], inplace=True)
df_lib.rename(columns={'Suburb - Incident':'Suburb', 'Postcode - Incident':'Postcode', 'Offence Level 2 Description':'Offence', 'Offence count':'Count'}, inplace=True)
df_lib=df_lib.dropna()
df_lib=df_lib.sort_index().truncate(before='2012-01-01', after='2020-03-31')
pivot_lib=pd.pivot_table(df_lib, values=['Count'], index=(['Reported Date']), columns=['Offence'], aggfunc=np.sum)
pivot_lib.plot(figsize=(25,8), title='Offences Reported from 2012 to present')

# 4. Function I

In [ ]:
def SA_Suburbwise_MontlyAvg_performance (offence):
    
#   1.  Read file

    df_lib=pd.read_csv('Data/Crime_data.csv', parse_dates=True)
    SA_suburbs=gpd.read_file('Data/GreaterAdelaideRegion_Suburbs.shp')
    
#   2.  Clean Data

    df_lib.drop(columns=['Unnamed: 0','Offence Level 1 Description','Offence Level 3 Description'], inplace=True)
    df_lib.rename(columns={'Suburb - Incident':'Suburb', 'Postcode - Incident':'Postcode', 'Offence Level 2 Description':'Offence', 'Offence count':'Count'}, inplace=True)
    df_lib=df_lib.dropna()
    df_lib['Day'],df_lib['Month'],df_lib['Year']= df_lib['Reported Date'].str.split('/',2).str
    df_lib['Month']=df_lib['Month'].astype(int)
    df_lib['Month'] = df_lib['Month'].apply(lambda x: calendar.month_abbr[x])#converting to string from 01 to Jan
    
#   3.  Data filtering for paramters in the function

    df_year_offence=df_lib[(df_lib['Offence']==offence)]

#   4. Grouping and Calculation

    df_year_offence_monthly=df_year_offence.groupby(['Year','Suburb', 'Month']).agg({'Count':np.sum})#Monthly total offences recorded
    df_year_offence_monthly=df_year_offence_monthly.reset_index()
    
    pivot=pd.pivot_table(df_year_offence_monthly, values=['Count'], index=['Year','Suburb'], aggfunc=np.mean)#Monthly average offences recorded each year
    pivot=pivot.reset_index()
    
#   5. Interactive Map

#   5.1. Sub-filter by Year using another function inside the function

    def json_data(selectedYear):
        yr = selectedYear
        df_yr = pivot[pivot['Year'] == yr]
        merged = SA_suburbs.merge(pivot, left_on = 'suburb', right_on = 'Suburb', how = 'left')#Merging csv data to shp
        merged_json = json.loads(merged.to_json())#converting to json file to make it interactive
        json_data = json.dumps(merged_json)
        return json_data
    geosource = GeoJSONDataSource(geojson = json_data(2020))
    
#   6. Plotting
    
    palette = brewer['YlGnBu'][8]
    palette = palette[::-1]
    color_mapper = LinearColorMapper(palette = palette, low = 0, high = 350)
    hover = HoverTool(tooltips = [ ('Suburb','@suburb'),('Offence Count', '@Count'), ('Year','@Year')])#To display the data when cursor placed on a polygon
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')
    p = figure(title = 'Monthly average Recorded %s' %offence, plot_height = 800 , plot_width = 950, toolbar_location = None, tools=[hover])
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.patches('xs','ys', source = geosource,fill_color = {'field' :'Count', 'transform' : color_mapper}, line_color = 'black', line_width = 0.25, fill_alpha = 1)
    p.add_layout(color_bar, 'below')
    
    def update_plot(attr, old, new):#Callback function to update the plot
        yr = slider.value
        new_data = json_data(yr)
        geosource.geojson = new_data
        p.title.text = 'Suburb-wise %s Count for %s' %(offence, yr)
    
    slider = Slider(title = 'Year',start = 2011, end = 2020, step = 1, value = 2020)
    slider.on_change('value', update_plot)
    
    layout = column(p,widgetbox(slider))
    curdoc().add_root(layout)
    
    output_notebook()
    
    show(layout)
    
    print(pivot)
    print(df_year_offence_monthly[(df_year_offence_monthly['Suburb']=='ADELAIDE')])
SA_Suburbwise_MontlyAvg_performance ('THEFT AND RELATED OFFENCES')

# 5. Function II

In [ ]:
def suburb_crime_status (suburb, month, offence):
#   1.  Read file
    df_lib=pd.read_csv('Data/Crime_data.csv', parse_dates=True)
    
#   2.  Clean Data
    df_lib.drop(columns=['Unnamed: 0','Offence Level 1 Description','Offence Level 3 Description'], inplace=True)
    df_lib.rename(columns={'Suburb - Incident':'Suburb', 'Postcode - Incident':'Postcode', 'Offence Level 2 Description':'Offence', 'Offence count':'Count'}, inplace=True)
    df_lib=df_lib.dropna()

    df_lib['Day'],df_lib['Month'],df_lib['Year']= df_lib['Reported Date'].str.split('/',2).str
    df_lib['Month']=df_lib['Month'].astype(int)
    df_lib['Month'] = df_lib['Month'].apply(lambda x: calendar.month_abbr[x])
    
#   3.  Data filtering for paramters in the function
    df_suburb=df_lib[(df_lib['Suburb']==suburb)]
    df_suburb_month=df_suburb[(df_suburb['Month']==month)]
    df_suburb_month_offence=df_suburb_month[(df_suburb_month['Offence']==offence)]
#   4.  Calculation
    df_suburb_month_offence_sum=df_suburb_month_offence.groupby(['Year']).agg({'Count':np.sum})
    df_suburb_month_offence_sum=df_suburb_month_offence_sum.reset_index()
    
#   5.  plotting
    
    f, ax = plt.subplots(figsize=(25,8))
    sns.barplot( x=df_suburb_month_offence_sum['Year'], y=df_suburb_month_offence_sum['Count'])
    plt.title('%s Crime status of %s for %s' %(offence, suburb, month), fontsize=18)
    
    
suburb_crime_status ('ADELAIDE', 'Mar', 'THEFT AND RELATED OFFENCES')